https://datahack.analyticsvidhya.com/contest/wns-triange-hackquest/

In [2]:
!conda install -y gdown
!gdown 1yVKnEnZeVZbQmKYG_nf1YD0SKFOQwdsu
!gdown 1DAuF_rbhXWk6A95orHObmNoxXHs402no

Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    filelock-3.13.1            |     pyhd8ed1ab_0          15 KB  conda-forge
    gdown-5.1.0                |     pyhd8ed1ab_0          21 KB  conda-forge
    openssl-3.2.1              |       hd590300_0         2.7 MB  conda-forge
    scip

In [3]:
!unzip -o train.zip
!unzip -o test.zip

!pip install pandas numpy scikit-learn imbalanced-learn

In [5]:
import os
import shutil
import random
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from sklearn.utils import resample
from collections import defaultdict
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2024-02-12 08:38:23.869153: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 08:38:23.869278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 08:38:24.003116: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
csv_file_path = '/kaggle/working/train/train.csv'
image_folder_path = '/kaggle/working/train/images'

In [7]:
min_width = float('inf')
min_height = float('inf')

for filename in os.listdir(image_folder_path):
    if filename.endswith(".jpg"):
        img_path = os.path.join(image_folder_path, filename)
        img = Image.open(img_path)
        width, height = img.size
        min_width = min(min_width, width)
        min_height = min(min_height, height)
print("Minimum width:", min_width)
print("Minimum height:", min_height)

Minimum width: 800
Minimum height: 600


In [ ]:
df = pd.read_csv(csv_file_path)
labels_dict = dict(zip(df['filename'], df['label']))
class_0_images = [img for img, label in labels_dict.items() if label == 0]
class_1_images = [img for img, label in labels_dict.items() if label == 1]

class_0_images_data = []
class_1_images_data = []
class_1_labels = []

min_width = float('inf')
min_height = float('inf')

for image_name in class_0_images:
    img_path = os.path.join(image_folder_path, image_name)
    img = Image.open(img_path)
    width, height = img.size
    min_width = min(min_width, width)
    min_height = min(min_height, height)
    img = img.resize((min_width, min_height))
    class_0_images_data.append(img)

for image_name in class_1_images:
    img_path = os.path.join(image_folder_path, image_name)
    img = Image.open(img_path)
    width, height = img.size
    min_width = min(min_width, width)
    min_height = min(min_height, height)
    img = img.resize((min_width, min_height))
    class_1_images_data.append(img)
    class_1_labels.append(labels_dict[image_name])

datagen = ImageDataGenerator(
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest'
)

oversampled_class_1_images = resample(class_1_images, replace=True, n_samples=len(class_1_images)*2)
for image_name in oversampled_class_1_images:
    img_path = os.path.join(image_folder_path, image_name)
    img = Image.open(img_path)
    width, height = img.size
    min_width = min(min_width, width)
    min_height = min(min_height, height)
    img = img.resize((min_width, min_height))
    class_1_images_data.append(img)
    class_1_labels.append(labels_dict[image_name])

    augment_iter = datagen.flow(np.expand_dims(img, axis=0), batch_size=1)
    for _ in range(2):
        augmented_img = Image.fromarray(augment_iter.next()[0].astype(np.uint8))
        class_1_images_data.append(augmented_img)
        class_1_labels.append(labels_dict[image_name])

balanced_images_data = class_0_images_data + class_1_images_data
balanced_labels = [0] * len(class_0_images_data) + class_1_labels

x = np.array([np.array(img) for img in balanced_images_data])
y = np.array(balanced_labels)

In [ ]:
np.savez('train_data.npz', balanced_images_data=balanced_images_data)

In [ ]:
x = np.array([np.array(img) for img in balanced_images_data])
y = np.array(balanced_labels)

In [ ]:
len(x)

In [ ]:
import os
folder_path = "/kaggle/working/train/images"
files = os.listdir(folder_path)
len(files)

In [ ]:
np.savez('train_data.npz', x=x, y=y)

In [ ]:
data = np.load('/kaggle/working/train_data.npz')

x = data['x']
y = data['y']

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=24, kernel_size=3, activation='relu', input_shape=(500, 500, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(images, train_labels_resampled, epochs=8, batch_size=24, validation_split=0.2)

In [ ]:
image_folder_path = '/kaggle/working/test/images'
for image_name in image_folder_path:
    print(image_name)

In [ ]:
image_folder_path = '/kaggle/working/test/images'

test = []

for image_name in image_folder_path:
    img_path = os.path.join(image_folder_path, image_name)
    img = Image.open(img_path)
    img = img.resize((min_width, min_height))
    test.append(img)

test = np.array([np.array(img) for img in test])

In [ ]:
y_pred = model.predict(test)

In [ ]:
filenames = generator.filenames
image_ids = [os.path.splitext(os.path.basename(filename))[0] for filename in filenames]
threshold = 0.5
binary_predictions = (y_pred >= threshold).astype(int)
data = {'image_id': image_ids, 'label': binary_predictions.flatten()}
df = pd.DataFrame(data)
csv_filename = 'predictions.csv'
df.to_csv(csv_filename, index=False)
print(f"Predictions saved to {csv_filename}.")